<a href="https://www.kaggle.com/code/godwinbentummensah/epl-standings-predictions?scriptVersionId=197824537" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import learning_curve
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install kaggle


# **Previous Season Results**

#### *Note: UEFA started allowing the 4th place team to go straight to the Champions League group stage in the 2018-19 season.

### **Get previous season results into dataframe**

In [ ]:
#create dataframe
past_seasons_df = pd.read_csv('/kaggle/input/eplepl/epl.csv')

past_seasons_df.head(1000)




In [ ]:
past_seasons_df.info()

# **Data Quality Report**

In [ ]:
data_types = pd.DataFrame(past_seasons_df.dtypes, columns=['Data Type'])
missing_data = pd.DataFrame(past_seasons_df.isnull().sum(), columns=['Missing Values'])

unique_values = pd.DataFrame(columns=['Unique Values'])
for row in list(past_seasons_df.columns.values):
    unique_values.loc[row] = [past_seasons_df[row].nunique()]

maximum_values = pd.DataFrame(columns=['Max Value'])
for row in list(past_seasons_df.columns.values):
    maximum_values.loc[row] = [past_seasons_df[row].max()]

minimum_values = pd.DataFrame(columns=['Min Value'])
for row in list(past_seasons_df.columns.values):
    minimum_values.loc[row] = [past_seasons_df[row].min()]
    
dq_report = data_types.join(missing_data).join(unique_values).join(maximum_values).join(minimum_values)
dq_report


In [ ]:
#top four of each season
seasons_top_four = past_seasons_df.groupby('Season').head(4)

#bottom three of each season
seasons_bottom_three = past_seasons_df.groupby('Season').tail(3)

In [ ]:
seasons_top_four

In [ ]:
seasons_bottom_three

In [ ]:
# save qualification/relegation column to new dataframe, and drop from original dataframe
seasons_results_df = past_seasons_df[['Qualification or relegation']].copy()
past_seasons_df = past_seasons_df.drop(columns=['Qualification or relegation'])


past_seasons_df

In [ ]:
seasons_results_df.head(20)


# **Data Analysis**

### **Correlation**

In [ ]:
past_stats = past_seasons_df[['W','D','L','GF','GA','GD','Pts']].copy()
#past_stats
sns.heatmap(past_stats.corr(), annot=True, cmap='RdYlGn', center = 0)

In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


All the stats clearly correlate with the amount of points a team earns.

Season Analysis

In [ ]:
winners = (past_seasons_df[past_seasons_df.Pos == 1])
winners

In [ ]:
most_times_w = winners.Team.value_counts()
most_times_w
sns.catplot(y = 'Team', kind='count', data = winners, aspect = 2.5, order = most_times_w.index, palette = 'summer')
plt.title('Champions between 2000-2022', fontsize = 20)
plt.xlabel('Titles')

In [ ]:
best_campaings = winners.sort_values('Pts',ascending=False)
best_campaings.head(1)

In [ ]:
losers = (past_seasons_df[past_seasons_df.Pos >= 18])
relegations = losers.Team.value_counts()

In [ ]:
plt.figure(figsize=(12, 12))
sns.barplot(x=relegations, y=relegations.index, palette = 'autumn')
plt.title('Relegations between 2000-2022');
plt.xlabel('Relegations');
plt.ylabel('Team');

In [ ]:
points_df = past_seasons_df.groupby('Team').agg({'Pts': 'sum'})
matches_df = past_seasons_df.groupby('Team').agg({'Pld':'sum'})
points_df.sort_values('Pts', ascending=False)
matches_df.sort_values('Pld', ascending=False)

effectivity_df = pd.merge(points_df, matches_df, on='Team')
effectivity_df["Total Points"] = effectivity_df['Pld']*3
effectivity_df["Eff"] = effectivity_df['Pts']/effectivity_df['Total Points']

eff_df = effectivity_df.copy()
del eff_df['Pts']
del eff_df['Pld']
del eff_df['Total Points']

eff_df = eff_df.sort_values("Eff", ascending = True)

te = eff_df.plot(kind='barh',figsize=(12,14), color='xkcd:sky blue')
plt.title('Team Effectiveness between 2000-2022',fontsize=20)
plt.ylabel('Team',fontsize=20)
plt.xlabel('Effectivity',fontsize=20);
te.set_xlim(0.0, 1)


### **Typical Stats of Top 4 and Bottom 3 Teams**

In [ ]:
top_four = seasons_top_four[['W','D','L','GF','GA','GD','Pts']].copy()
bottom_three = seasons_bottom_three[['W','D','L','GF','GA','GD','Pts']].copy()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches  # Import mpatches

# Visual parameters
t4_color = '#3d195b'
b3_color = '#c9d6df'
b_color = '#f7f7f7'
seaborn_color = [b3_color, t4_color]
sns.set_palette(sns.color_palette(seaborn_color))

# Creating legends
t4_legend = mpatches.Patch(color=t4_color, label='Top 4', edgecolor='black')
b3_legend = mpatches.Patch(color=b3_color, label='Bottom 3', edgecolor='black')

# Add legends to your plot later on
plt.legend(handles=[t4_legend, b3_legend], loc='center')  # Example location


#### **Wins, Draws, and Losses Distributions**

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20,25))
cols = ['W','D','L']

for i, col in enumerate(cols):
    t4_index = top_four[col].value_counts().index
    t4_values = top_four[col].value_counts().values
    b3_index = bottom_three[col].value_counts().index
    b3_values = bottom_three[col].value_counts().values
    
    ax.flatten()[i].bar(x=t4_index, height=t4_values/1.08, color=t4_color, edgecolor='red', alpha=1, linewidth=2, label='Top 4')
    ax.flatten()[i].bar(x=b3_index, height=b3_values/4.38, color=b3_color, edgecolor='red', alpha=0.8, linewidth=2, label='Bottom 3')
    
    ax.flatten()[i].grid(which='major', axis='x', zorder=0, alpha=0.3)
    ax.flatten()[i].grid(which='major', axis='y', zorder=0, alpha=0.3)
    
    ax.flatten()[i].set_xticks(np.arange(0, 38, 1))
    ax.flatten()[i].set_xticklabels(ax.flatten()[i].get_xticks(), fontsize=15)
    ax.flatten()[i].set_yticklabels(ax.flatten()[i].get_yticks(), fontsize=15)
    ax.flatten()[i].set_xlabel('Number of games', fontsize=20)
    ax.flatten()[i].set_ylabel('%', fontsize=20)
    ax.flatten()[i].set_title(col.upper(), weight='bold', fontsize=20)
    ax.flatten()[i].set_facecolor(b_color)
    
    ax.flatten()[i].spines['top'].set_visible(False)
    ax.flatten()[i].spines['right'].set_visible(False)
    ax.flatten()[i].spines['bottom'].set_linewidth(4)
    ax.flatten()[i].spines['left'].set_linewidth(4)
    
    if i == 0:
        ax.flatten()[i].legend(loc=1, shadow=True, fontsize=12)

In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25,20))
col = 'Pts'

t4_index = top_four[col].value_counts().index
t4_values = top_four[col].value_counts().values
b3_index = bottom_three[col].value_counts().index
b3_values = bottom_three[col].value_counts().values

ax.bar(x=t4_index, height=t4_values/1.08, color=t4_color, edgecolor='black', alpha=1, linewidth=2.5, label='Top 4')
ax.bar(x=b3_index, height=b3_values/4.38, color=b3_color, edgecolor='black', alpha=0.8, linewidth=2.5, label='Bottom 3')

ax.grid(which='major', axis='x', zorder=0, alpha=0.3)
ax.grid(which='major', axis='y', zorder=0, alpha=0.3)

ax.set_xticks(np.arange(0, 110, 5))
ax.set_title(col.upper(), weight='bold', fontsize=25)
ax.set_ylabel('%', fontsize=20)
ax.set_xticklabels(ax.get_xticks(), fontsize=20)
ax.set_yticklabels(ax.get_yticks(), fontsize=15)
ax.xaxis.set_tick_params(labelsize=14)
ax.set_facecolor(b_color)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(4)
ax.spines['left'].set_linewidth(4)

ax.legend(loc=1, shadow=True, fontsize=18)

fig.set_facecolor(b_color)
fig.tight_layout(pad=5)
plt.show()

In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


#### **Points Distribution**

#### **GF and GA Distributions**

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(25,20))
cols = ['GF','GA']

for i, col in enumerate(cols):
    t4_index = top_four[col].value_counts().index
    t4_values = top_four[col].value_counts().values
    b3_index = bottom_three[col].value_counts().index
    b3_values = bottom_three[col].value_counts().values
    
    ax.flatten()[i].bar(x=t4_index, height=t4_values/1.08, color=t4_color, edgecolor='black', alpha=1, linewidth=1.5, label='Top 4')
    ax.flatten()[i].bar(x=b3_index, height=b3_values/4.38, color=b3_color, edgecolor='black', alpha=0.8, linewidth=1.5, label='Bottom 3')
    
    ax.flatten()[i].grid(which='major', axis='x', zorder=0, alpha=0.3)
    ax.flatten()[i].grid(which='major', axis='y', zorder=0, alpha=0.3)
    
    ax.flatten()[i].set_xticks(np.arange(0, 110, 5))
    ax.flatten()[i].set_title(col.upper(), weight='bold', fontsize=20)
    ax.flatten()[i].set_ylabel('%', fontsize=20)
    ax.flatten()[i].set_facecolor(b_color)
    
    ax.flatten()[i].spines['top'].set_visible(False)
    ax.flatten()[i].spines['right'].set_visible(False)
    ax.flatten()[i].spines['bottom'].set_linewidth(4)
    ax.flatten()[i].spines['left'].set_linewidth(4)
    
    if i == 0:
        ax.flatten()[i].legend(loc=1, shadow=True, fontsize=14)

fig.set_facecolor(b_color)
fig.set_edgecolor('black')
fig.tight_layout(pad=5)
plt.show()

In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


In [ ]:
past_seasons_df.loc[(past_seasons_df['Pos'] == 4) & (past_seasons_df['GD'] < 0)]

Everton is the only team to finish in the top four with a negative GD from the seasons covered in provided data.

# **Train Model With Data**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Define the ANN model
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=input_dim))
    model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train models and print evaluation metrics
def train(X, y, X_valid, y_valid):
    knn = KNeighborsClassifier(n_neighbors=5)
    dt  = DecisionTreeClassifier(max_depth=5)
    rf = RandomForestClassifier(n_estimators=100, max_features=5)
    ann = KerasClassifier(build_fn=lambda: build_ann(X.shape[1]), epochs=100, batch_size=10, verbose=0)
    svm = SVC(kernel='linear')
    
    # Fit models
    rf.fit(X, y)
    ann.fit(X, y)
    knn.fit(X, y)
    dt.fit(X, y)
    svm.fit(X, y)

    # Make predictions
    ann_pred = (ann.predict(X_valid) > 0.5).astype(int).ravel()
    svm_pred = svm.predict(X_valid)
    knn_pred = knn.predict(X_valid)
    dt_pred = dt.predict(X_valid)
    rf_pred = rf.predict(X_valid)

    # Print evaluation
    print(f'K-Nearest Neighbors: {round(accuracy_score(y_valid, knn_pred), 5) * 100:.0f}%')
    print(f'Random Forest: {round(accuracy_score(y_valid, rf_pred), 5) * 100:.0f}%')
    print(f'Artificial Neural Network: {round(accuracy_score(y_valid, ann_pred), 5) * 100:.0f}%')
    print(f'Support Vector Machine: {round(accuracy_score(y_valid, svm_pred), 5) * 100:.0f}%')
    print(f'Decision Tree: {round(accuracy_score(y_valid, dt_pred), 5) * 100:.0f}%')

    return [rf, ann, svm, dt, knn]

    print(f'K-Nearest Neighbors: {round(precision_score(y_valid, knn_pred), 5) * 100:.0f}%')
    print(f'Random Forest: {round(precision_score(y_valid, rf_pred), 5) * 100:.0f}%')
    print(f'Artificial Neural Network: {round(precision_score(y_valid, ann_pred), 5) * 100:.0f}%')
    print(f'Support Vector Machine: {round(precision_score(y_valid, svm_pred), 5) * 100:.0f}%')
    print(f'Decision Tree: {round(precision_score(y_valid, dt_pred), 5) * 100:.0f}%')

    return [rf, ann, svm, dt, knn]

    print(f'K-Nearest Neighbors: {round(recall_score(y_valid, knn_pred), 5) * 100:.0f}%')
    print(f'Random Forest: {round(recall_score(y_valid, rf_pred), 5) * 100:.0f}%')
    print(f'Artificial Neural Network: {round(recall_score(y_valid, ann_pred), 5) * 100:.0f}%')
    print(f'Support Vector Machine: {round(recall_score(y_valid, svm_pred), 5) * 100:.0f}%')
    print(f'Decision Tree: {round(recall_score(y_valid, dt_pred), 5) * 100:.0f}%')

    return [rf, ann, svm, dt, knn]

    print(f'K-Nearest Neighbors: {round(f1_score(y_valid, knn_pred), 5) * 100:.0f}%')
    print(f'Random Forest: {round(f1_score(y_valid, rf_pred), 5) * 100:.0f}%')
    print(f'Artificial Neural Network: {round(f1_score(y_valid, ann_pred), 5) * 100:.0f}%')
    print(f'Support Vector Machine: {round(f1_score(y_valid, svm_pred), 5) * 100:.0f}%')
    print(f'Decision Tree: {round(f1_score(y_valid, dt_pred), 5) * 100:.0f}%')

    return [rf, ann, svm, dt, knn]

    # Plot learning curves for different models
def plot_learning(X, y):
    rf = RandomForestClassifier(n_estimators=100, max_features=7)
    ann = KerasClassifier(build_fn=lambda: build_ann(X.shape[1]), epochs=100, batch_size=10, verbose=0)
    svm = SVC(kernel='linear')
    knn = KNeighborsClassifier(n_neighbors=5)
    dt  = DecisionTreeClassifier(max_depth=5)

    models = [rf, knn, ann, svm, dt]
    model_names = ['Random Forest', 'K-Nearest Neighbors', 'Artificial Neural Network', 'Support Vector Machine', 'Decision Tree']

    fig, ax = plt.subplots(1, 5, figsize=(24, 5))

    for i in range(5):
        train_sizes, train_scores, test_scores = learning_curve(models[i], X, y, cv=5, n_jobs=-1)
        ax.flatten()[i].plot(train_sizes, np.mean(train_scores, axis=1), 'o-', color='blue', label='Training')
        ax.flatten()[i].plot(train_sizes, np.mean(test_scores, axis=1), 'o-', color='brown', label='Cross-Validation')
        ax.flatten()[i].legend()
        ax.flatten()[i].set_title(f'{model_names[i]} Learning Curve')
        ax.flatten()[i].set_xlabel('Training Size')
        ax.flatten()[i].set_ylabel('Score')

    plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Define the ANN model
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=input_dim))
    model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train models and print evaluation metrics
def train(X, y, X_valid, y_valid):
    knn = KNeighborsClassifier(n_neighbors=5)
    dt  = DecisionTreeClassifier(max_depth=5)
    rf = RandomForestClassifier(n_estimators=100, max_features=5)
    ann = KerasClassifier(build_fn=lambda: build_ann(X.shape[1]), epochs=100, batch_size=10, verbose=0)
    svm = SVC(kernel='linear')

    # Fit models
    rf.fit(X, y)
    ann.fit(X, y)
    knn.fit(X, y)
    dt.fit(X, y)
    svm.fit(X, y)

    # Make predictions
    ann_pred = (ann.predict(X_valid) > 0.5).astype(int).ravel()
    svm_pred = svm.predict(X_valid)
    knn_pred = knn.predict(X_valid)
    dt_pred = dt.predict(X_valid)
    rf_pred = rf.predict(X_valid)

    # Accuracy
    print(f'K-Nearest Neighbors Accuracy: {accuracy_score(y_valid, knn_pred):.2%}')
    print(f'Random Forest Accuracy: {accuracy_score(y_valid, rf_pred):.2%}')
    print(f'Artificial Neural Network Accuracy: {accuracy_score(y_valid, ann_pred):.2%}')
    print(f'Support Vector Machine Accuracy: {accuracy_score(y_valid, svm_pred):.2%}')
    print(f'Decision Tree Accuracy: {accuracy_score(y_valid, dt_pred):.2%}')

    # Precision
    print(f'K-Nearest Neighbors Precision: {precision_score(y_valid, knn_pred):.2%}')
    print(f'Random Forest Precision: {precision_score(y_valid, rf_pred):.2%}')
    print(f'Artificial Neural Network Precision: {precision_score(y_valid, ann_pred):.2%}')
    print(f'Support Vector Machine Precision: {precision_score(y_valid, svm_pred):.2%}')
    print(f'Decision Tree Precision: {precision_score(y_valid, dt_pred):.2%}')

    # Recall
    print(f'K-Nearest Neighbors Recall: {recall_score(y_valid, knn_pred):.2%}')
    print(f'Random Forest Recall: {recall_score(y_valid, rf_pred):.2%}')
    print(f'Artificial Neural Network Recall: {recall_score(y_valid, ann_pred):.2%}')
    print(f'Support Vector Machine Recall: {recall_score(y_valid, svm_pred):.2%}')
    print(f'Decision Tree Recall: {recall_score(y_valid, dt_pred):.2%}')

    # F1 Score
    print(f'K-Nearest Neighbors F1 Score: {f1_score(y_valid, knn_pred):.2%}')
    print(f'Random Forest F1 Score: {f1_score(y_valid, rf_pred):.2%}')
    print(f'Artificial Neural Network F1 Score: {f1_score(y_valid, ann_pred):.2%}')
    print(f'Support Vector Machine F1 Score: {f1_score(y_valid, svm_pred):.2%}')
    print(f'Decision Tree F1 Score: {f1_score(y_valid, dt_pred):.2%}')

    return [rf, ann, svm, dt, knn]

# Plot learning curves for different models
def plot_learning(X, y):
    rf = RandomForestClassifier(n_estimators=100, max_features=7)
    ann = KerasClassifier(build_fn=lambda: build_ann(X.shape[1]), epochs=100, batch_size=10, verbose=0)
    svm = SVC(kernel='linear')
    knn = KNeighborsClassifier(n_neighbors=5)
    dt  = DecisionTreeClassifier(max_depth=5)

    models = [rf, knn, ann, svm, dt]
    model_names = ['Random Forest', 'K-Nearest Neighbors', 'Artificial Neural Network', 'Support Vector Machine', 'Decision Tree']

    fig, ax = plt.subplots(1, 5, figsize=(24, 5))

    for i in range(5):
        train_sizes, train_scores, test_scores = learning_curve(models[i], X, y, cv=5, n_jobs=-1)
        ax.flatten()[i].plot(train_sizes, np.mean(train_scores, axis=1), 'o-', color='blue', label='Training')
        ax.flatten()[i].plot(train_sizes, np.mean(test_scores, axis=1), 'o-', color='brown', label='Cross-Validation')
        ax.flatten()[i].legend()
        ax.flatten()[i].set_title(f'{model_names[i]} Learning Curve')
        ax.flatten()[i].set_xlabel('Training Size')
        ax.flatten()[i].set_ylabel('Score')

    plt.show()


In [ ]:
past_seasons = past_seasons_df.drop('Team', axis=1)
past_seasons['Top 4'] = 0
past_seasons['Bottom 3'] = 0

past_seasons.loc[past_seasons['Pos'] <= 4, 'Top 4'] = 1
past_seasons.loc[past_seasons['Pos'] >= 18, 'Bottom 3'] = 1
p_seasons = past_seasons.drop(columns=['Pos','Season','Pld'], axis=1)

p_seasons


In [ ]:
# Split Train and Test sets: first is for predicting top 4, second is for predicting bottom three
X_seasons_train, X_seasons_test, y_seasons_train, y_seasons_test = train_test_split(p_seasons.drop(columns=['Top 4','Bottom 3'], axis=1), 
                                                                        p_seasons['Top 4'], 
                                                                        test_size=0.2, 
                                                                        random_state=2023, 
                                                                        stratify=p_seasons['Top 4'])

X_seasons2_train, X_seasons2_test, y_seasons2_train, y_seasons2_test = train_test_split(p_seasons.drop(columns=['Top 4','Bottom 3'], axis=1), 
                                                                        p_seasons['Bottom 3'], 
                                                                        test_size=0.2, 
                                                                        random_state=2023, 
                                                                        stratify=p_seasons['Bottom 3'])

# Split Test set for validation set
X_seasons_valid, X_seasons_test, y_seasons_valid, y_seasons_test = train_test_split(X_seasons_test, 
                                                                        y_seasons_test, 
                                                                        test_size=0.2, 
                                                                        random_state=2023, 
                                                                        stratify=y_seasons_test)

X_seasons2_valid, X_seasons2_test, y_seasons2_valid, y_seasons2_test = train_test_split(X_seasons2_test, 
                                                                        y_seasons2_test, 
                                                                        test_size=0.2, 
                                                                        random_state=2023, 
                                                                        stratify=y_seasons2_test)


In [ ]:
X_seasons_test

### **Top 4 Learning**

In [ ]:
import pandas as pd

# Ensure current_season is defined (replace with your actual DataFrame)
current_season = pd.DataFrame({
    'Team','Pld','W','L','D','GF','GA','GD','Pts'
})

# Check if 'missing_column' exists, if not, add it with a default value of 0
if 'missing_column' not in current_season.columns:
    current_season['missing_column'] = 0

print(current_season)


In [ ]:
print(current_season.shape)  # Should match the shape of training data (without the target column)


In [ ]:
 t4_rf, t4_ann, t4_svm, t4_knn, t4_dt = train(X_seasons_train, y_seasons_train, X_seasons_valid, y_seasons_valid)

In [ ]:
plot_learning(X_seasons_train, y_seasons_train)

In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


### **Bottom 3 Learning**

In [ ]:
b3_rf, b3_ann, b3_svm, b3_knn, b3_dt = train(X_seasons2_train, y_seasons2_train, X_seasons2_valid, y_seasons2_valid)

In [ ]:
plot_learning(X_seasons2_train, y_seasons2_train)

In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


# **Predicting Current Season Standings**

#### Current season standings dataframe

In [ ]:
#create dataframe for current season; edited csv from https://www.footballwebpages.co.uk/premier-league/league-table to match other dataframes
current_season_df = pd.read_csv('//kaggle/input/newepl/eplcurrent.csv')
current_season_df.index += 1

current_season_df

In [ ]:
current_season = current_season_df.copy()
current_season = current_season.reset_index(drop=True)
current_season['Top 4'] = 0
current_season['Bottom 3'] = 0

current_season.loc[current_season['Pos'] <= 4, 'Top 4'] = 1
current_season.loc[current_season['Pos'] >= 18, 'Bottom 3'] = 1

y_test_t4 = current_season['Top 4'].copy()
y_test_b3 = current_season['Bottom 3'].copy()

teams = current_season[['Pos','Team']].copy()

In [ ]:
current_season = current_season.drop(columns=['Season','Pld','Pos','Team','Top 4', 'Bottom 3'])
current_season

In [ ]:
y_test_t4

In [ ]:
y_test_b3

### Predicting top 4

In [ ]:
# Accuracy
print((f't4_ann: {round(accuracy_score(y_test_t4, t4_ann.predict(current_season)) * 100, 2)}% \n'
       f't4_rf: {round(accuracy_score(y_test_t4, t4_rf.predict(current_season)) * 100, 2)}% \n'
       f't4_svm: {round(accuracy_score(y_test_t4, t4_svm.predict(current_season)) * 100, 2)}% \n'
       f't4_knn: {round(accuracy_score(y_test_t4, t4_knn.predict(current_season)) * 100, 2)}% \n'
       f't4_dt: {round(accuracy_score(y_test_t4, t4_dt.predict(current_season)) * 100, 2)}%'))

# Precision
print((f't4_ann: {round(precision_score(y_test_t4, t4_ann.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_rf: {round(precision_score(y_test_t4, t4_rf.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_svm: {round(precision_score(y_test_t4, t4_svm.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_knn: {round(precision_score(y_test_t4, t4_knn.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_dt: {round(precision_score(y_test_t4, t4_dt.predict(current_season), average="binary") * 100, 2)}%'))

# Recall
print((f't4_ann: {round(recall_score(y_test_t4, t4_ann.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_rf: {round(recall_score(y_test_t4, t4_rf.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_svm: {round(recall_score(y_test_t4, t4_svm.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_knn: {round(recall_score(y_test_t4, t4_knn.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_dt: {round(recall_score(y_test_t4, t4_dt.predict(current_season), average="binary") * 100, 2)}%'))

# F1 Score
print((f't4_ann: {round(f1_score(y_test_t4, t4_ann.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_rf: {round(f1_score(y_test_t4, t4_rf.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_svm: {round(f1_score(y_test_t4, t4_svm.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_knn: {round(f1_score(y_test_t4, t4_knn.predict(current_season), average="binary") * 100, 2)}% \n'
       f't4_dt: {round(f1_score(y_test_t4, t4_dt.predict(current_season), average="binary") * 100, 2)}%'))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Metrics for the algorithms
models = ['ANN', 'Random Forest', 'SVM', 'KNN', 'Decision Tree']
accuracy = [95.00, 100.00, 95.00, 100.00, 100.00]
precision = [100.00, 100.00, 100.00, 100.00, 100.00]
recall = [75.00, 100.00, 75.00, 100.00, 100.00]
f1_score = [85.71, 100.00, 85.71, 100.00, 100.00]

# Create an array for bar positions
x = np.arange(len(models))

# Set width of the bars
width = 0.2

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(x - width*1.5, accuracy, width, label='Accuracy', color='skyblue')
ax.bar(x - width/2, precision, width, label='Precision', color='lightgreen')
ax.bar(x + width/2, recall, width, label='Recall', color='salmon')
ax.bar(x + width*1.5, f1_score, width, label='F1 Score', color='orange')

# Add labels, title, and ticks
ax.set_xlabel('Algorithms')
ax.set_ylabel('Percentage (%)')
ax.set_title('Comparison of ML Algorithms Performance Metrics')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


In [ ]:
# Calculate average prediction only for models that support predict_proba
probas = {
    'ANN': t4_ann.predict_proba(current_season)[:, 1],
    'RF': t4_rf.predict_proba(current_season)[:, 1],
    'SVM': t4_svm.decision_function(current_season),  # Use decision_function as an alternative
    'KNN': t4_knn.predict_proba(current_season)[:, 1],
    'DT': t4_dt.predict_proba(current_season)[:, 1]
}

# Convert decision_function to probabilities for SVM (use sigmoid function if needed)
from scipy.special import expit  # Sigmoid function
probas['SVM'] = expit(probas['SVM'])

# Average the probabilities
avg_pred_t4 = (probas['ANN'] + 
               probas['RF'] +
               probas['SVM'] + 
               probas['KNN'] +
               probas['DT']) / 5
avg_pred_t4


In [ ]:
import pandas as pd

# Assuming avg_pred_t4 is defined and you want to use it
df_results = pd.concat([
    teams,
    pd.Series(avg_pred_t4, name='ensemble_result'),
    pd.Series(y_test_t4, name='Top 4')
], axis=1)

print(df_results.head())


### Predicting bottom three

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming the following predictions and true values:
# y_test_b3 is your true labels
# b3_ann, b3_rf, b3_svm, b3_knn, b3_dt are your models

# Accuracy
print((f'b3_ann: {round(accuracy_score(y_test_b3, b3_ann.predict(current_season)) * 100, 2)}% \n'
       f'b3_rf: {round(accuracy_score(y_test_b3, b3_rf.predict(current_season)) * 100, 2)}% \n'
       f'b3_svm: {round(accuracy_score(y_test_b3, b3_svm.predict(current_season)) * 100, 2)}% \n'
       f'b3_knn: {round(accuracy_score(y_test_b3, b3_knn.predict(current_season)) * 100, 2)}% \n'
       f'b3_dt: {round(accuracy_score(y_test_b3, b3_dt.predict(current_season)) * 100, 2)}%'))

# Precision
print((f'b3_ann: {round(precision_score(y_test_b3, b3_ann.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_rf: {round(precision_score(y_test_b3, b3_rf.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_svm: {round(precision_score(y_test_b3, b3_svm.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_knn: {round(precision_score(y_test_b3, b3_knn.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_dt: {round(precision_score(y_test_b3, b3_dt.predict(current_season), average="binary") * 100, 2)}%'))

# Recall
print((f'b3_ann: {round(recall_score(y_test_b3, b3_ann.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_rf: {round(recall_score(y_test_b3, b3_rf.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_svm: {round(recall_score(y_test_b3, b3_svm.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_knn: {round(recall_score(y_test_b3, b3_knn.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_dt: {round(recall_score(y_test_b3, b3_dt.predict(current_season), average="binary") * 100, 2)}%'))

# F1 Score
print((f'b3_ann: {round(f1_score(y_test_b3, b3_ann.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_rf: {round(f1_score(y_test_b3, b3_rf.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_svm: {round(f1_score(y_test_b3, b3_svm.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_knn: {round(f1_score(y_test_b3, b3_knn.predict(current_season), average="binary") * 100, 2)}% \n'
       f'b3_dt: {round(f1_score(y_test_b3, b3_dt.predict(current_season), average="binary") * 100, 2)}%'))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Metrics for the algorithms
models = ['ANN', 'Random Forest', 'SVM', 'KNN', 'Decision Tree']
accuracy = [95.00, 100.00, 100.00, 95.00, 100.00]
precision = [75.00, 100.00, 100.00, 75.00, 100.00]
recall = [100.00, 100.00, 100.00, 100.00, 100.00]
f1_score = [85.71, 100.00, 100.00, 85.71, 100.00]

# Create an array for bar positions
x = np.arange(len(models))

# Set width of the bars
width = 0.2

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(x - width*1.5, accuracy, width, label='Accuracy', color='skyblue')
ax.bar(x - width/2, precision, width, label='Precision', color='lightgreen')
ax.bar(x + width/2, recall, width, label='Recall', color='salmon')
ax.bar(x + width*1.5, f1_score, width, label='F1 Score', color='orange')

# Add labels, title, and ticks
ax.set_xlabel('Algorithms')
ax.set_ylabel('Percentage (%)')
ax.set_title('Comparison of ML Algorithms Performance Metrics')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
plt.savefig('/kaggle/working/plot.png', format='png')


In [ ]:
from scipy.special import expit  # Sigmoid function

# Calculate probabilities
probas = {
    'ANN': b3_ann.predict_proba(current_season)[:, 1],
    'RF': b3_rf.predict_proba(current_season)[:, 1],
    'SVM': b3_svm.decision_function(current_season),  # Use decision_function as an alternative
    'KNN': b3_knn.predict_proba(current_season)[:, 1],
    'DT': b3_dt.predict_proba(current_season)[:, 1]
}

# Convert decision_function to probabilities for SVM
probas['SVM'] = expit(probas['SVM'])

# Average the probabilities
avg_pred_b3 = (probas['ANN'] + 
               probas['RF'] +
               probas['SVM'] + 
               probas['KNN'] +
               probas['DT']) / 5

# Display the average predictions
avg_pred_b3


In [ ]:
import pandas as pd

# Assuming avg_pred_b3 is defined and you want to use it
df_results = pd.concat([
    teams,
    pd.Series(avg_pred_b3, name='ensemble_result'),
    pd.Series(y_test_b3, name='Bottom 3')
], axis=1)

print(df_results)


# **Final Result**

In [ ]:
final_table = pd.concat([teams,pd.Series(current_season['Pts'], name='Points'),pd.Series(avg_pred_t4, name='t4_ensemble_result'),pd.Series(y_test_t4, name='Top 4'),
          pd.Series(avg_pred_b3, name='b3_ensemble_result'),pd.Series(y_test_b3, name='Bottom 3')], axis=1)

def highlight(s):
    if s['t4_ensemble_result'] > 0:
        return ['background-color: green'] * len(s)
    if s['b3_ensemble_result'] >= 0.8:
        return ['background-color: red'] * len(s)
    if s['b3_ensemble_result'] > 0:
        return ['background-color: orange'] * len(s)

    
final_table = final_table.style.apply(highlight, axis=1)
final_table

The models predicts most of the current top four and bottom three teams to remain the same by the end of the season (Liverpool is given higher probability than Manchester United, for top 4, while Everton is given a higher probability than ). Of course, a lot can change with about 7 matches left in the season. The model for predicting the bottom three also shows that every team from the 11th place (where Chelsea currently is) is still in danger of ending up in the bottom three.

## **Closing**